# Exercise 1 - Inspecting a Bitcoin address

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from pprint import pprint

import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api

## Notebook Setup

This notebook assumes that you already followed the instructions outlined in `README.md` and installed the [GraphSense Python API](https://github.com/graphsense/graphsense-python) library locally.

First, we must setup our notebook and establish a connection to some hosted GraphSense instance.

We connect to a GraphSense instance hosted by [Iknaio](https://www.ikna.io/) and enter our API key in the provided `config.json` configuration file. An API key will be provided during the tutorial. If you would like to get an API key later, drop an email to contact@iknaio.com

## Load host and API key from config

In [3]:
f = open('config.json')
config = json.load(f)
f.close()
config

{'graphsense': {'host': 'https://api.ikna.io/',
  'api_key': 'wnTGntS7x8SLJ7nDwyMSfHVQg0Y9XYP3'}}

## Configure GraphSense client

Now we configure the GraphSense Python library

In [4]:
configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

We can test if our libary is working by retrieving summary statistics on supported ledgers.

In [5]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    pprint(api_response)

{'currencies': [{'name': 'btc',
                 'no_address_relations': 6324455334,
                 'no_addresses': 999362679,
                 'no_blocks': 744618,
                 'no_entities': 453401567,
                 'no_labels': 989,
                 'no_tagged_addresses': 235777372,
                 'no_txs': 748154188,
                 'timestamp': 1657583636},
                {'name': 'bch',
                 'no_address_relations': 2252085027,
                 'no_addresses': 331665580,
                 'no_blocks': 748221,
                 'no_entities': 150204333,
                 'no_labels': 84,
                 'no_tagged_addresses': 13369862,
                 'no_txs': 360726433,
                 'timestamp': 1657495979},
                {'name': 'ltc',
                 'no_address_relations': 1273105685,
                 'no_addresses': 140510180,
                 'no_blocks': 2273844,
                 'no_entities': 61175737,
                 'no_labels': 64,
    

# Address API

We start by retrieving some summary statistics on the Internet Archive's donation address. The response basically corresponds to the data shown in the property box of the GraphSense dashboard.

In [6]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = addresses_api.AddressesApi(api_client)

    currency = "btc"
    address = '1Archive1n2C579dMsAu3iC6tWzuQJz8dN'
    include_tags = True

    try:
        resp_addr = api_instance.get_address(currency, address)
        pprint(resp_addr)
    except graphsense.ApiException as e:
        print("Exception when calling AddressesApi->get_address: %s\n" % e)

{'address': '1Archive1n2C579dMsAu3iC6tWzuQJz8dN',
 'balance': {'fiat_values': [{'code': 'eur', 'value': 25160.42},
                             {'code': 'usd', 'value': 25407.0}],
             'value': 127222273},
 'currency': 'btc',
 'entity': 2647117,
 'first_tx': {'height': 256314,
              'timestamp': 1378415426,
              'tx_hash': '04d92601677d62a985310b61a301e74870fa942c8be0648e16b1db23b996a8cd'},
 'in_degree': 6396,
 'last_tx': {'height': 744585,
             'timestamp': 1657555640,
             'tx_hash': 'b90126f8c685191330d230a512eaaba2c4a6890bbbb16040b5721e62c80f2431'},
 'no_incoming_txs': 4850,
 'no_outgoing_txs': 274,
 'out_degree': 293,
 'total_received': {'fiat_values': [{'code': 'eur', 'value': 2485713.0},
                                    {'code': 'usd', 'value': 2960953.0}],
                    'value': 41264476217},
 'total_spent': {'fiat_values': [{'code': 'eur', 'value': 2369647.5},
                                 {'code': 'usd', 'value': 2838626.75

In [7]:
print(f"Address {resp_addr.address} received {resp_addr.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_addr.in_degree} addresses.")

Address 1Archive1n2C579dMsAu3iC6tWzuQJz8dN received 2485713.0 EUR from 6396 addresses.


# Entities API

Next, we inspect the corresponding entity, which clusters other addresses that are likely conrolled by the owner of that address. The entity ID is contained in the address response.

In [8]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = entities_api.EntitiesApi(api_client)

    currency = "btc" 
    entity = resp_addr.entity 
    include_tags = True 

    try:
        resp_entity = api_instance.get_entity(currency, entity)
        pprint(resp_entity)
    except graphsense.ApiException as e:
        print("Exception when calling EntitiesApi->get_entity: %s\n" % e)

{'balance': {'fiat_values': [{'code': 'eur', 'value': 25160.42},
                             {'code': 'usd', 'value': 25407.0}],
             'value': 127222273},
 'best_address_tag': {'address': '1Archive1n2C579dMsAu3iC6tWzuQJz8dN',
                      'category': 'organization',
                      'confidence': 'service_data',
                      'confidence_level': 50,
                      'currency': 'BTC',
                      'entity': 2647117.0,
                      'is_cluster_definer': True,
                      'label': 'internet archive',
                      'lastmod': 1636675200,
                      'source': 'https://archive.org/donate/cryptocurrency',
                      'tagpack_creator': 'GraphSense Core Team',
                      'tagpack_is_public': True,
                      'tagpack_title': 'GraphSense Demo TagPack',
                      'tagpack_uri': 'demo.yaml'},
 'currency': 'btc',
 'entity': 2647117,
 'first_tx': {'height': 156529,
       

In [9]:
print(f"Entity {resp_entity.entity} likely controls {resp_entity.no_addresses} addresses, " +
      f"which received {resp_entity.total_received.fiat_values[0].value} EUR " + 
      f"from {resp_entity.in_degree} entities.")

Entity 2647117 likely controls 120 addresses, which received 2517154.0 EUR from 5179 entities.


# Entity Neighbors

Finally, we retrieve all neighbor entities receiving funds from our focus entity. Since it is more covenient to work with dataframes, we use the GraphSense Bulk API interface to retrieve a dataframe of receiving neighbor nodes.

In [10]:
import pandas as pd

data = {'entity': [resp_addr.entity]}

entityDF = pd.DataFrame.from_dict(data)

In [11]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = bulk_api.BulkApi(api_client)

    CURRENCY = "btc"
    operation = "list_entity_neighbors"
    body = {'entity': entityDF['entity'].to_list(),
            'direction': 'out',
            'include_labels': True}

    try:
        respDF = pd.read_csv(
                    api_instance.bulk_csv(CURRENCY, operation, body=body,
                                          num_pages=2,
                                          _preload_content=False))
        pprint(respDF)
    except graphsense.ApiException as e:
        print("Exception when calling BulkApi->bulk_csv: %s\n" % e)

   _error  _info  _request_entity  entity_balance_eur  entity_balance_usd  \
0     NaN    NaN          2647117        2.630444e+08        2.656222e+08   
1     NaN    NaN          2647117        3.490069e+04        3.524272e+04   

   entity_balance_value  entity_best_address_tag_abuse  \
0         1330069043059                            NaN   
1             176473408                            NaN   

      entity_best_address_tag_address entity_best_address_tag_category  \
0  1CsBZRX2CpCg2Bk1tDyCCMYoS8WgeSijCA                           entity   
1  1PEoUKNxTZsc5rFSQvQjeTVwDE9vEDCRWm                          service   

  entity_best_address_tag_confidence  ...  entity_total_received_value  \
0                           override  ...             5212581902400158   
1                       service_data  ...              133520860219149   

  entity_total_spent_eur  entity_total_spent_usd  entity_total_spent_value  \
0           1.186978e+10            1.388656e+10          52112518333

We filter the result and see the reltaions between known entity IDs. We also see two known receiving entities.

In [12]:
resultDF = respDF[['entity_entity', '_request_entity', 'labels', 'no_txs']] \
    .rename(columns={'entity_entity': 'src', '_request_entity': 'dst', 'labels': 'dst_label'})
resultDF

,src,dst,dst_label,no_txs
0,109577,2647117,"bitcoin-24.com,epay.info,supercluster",2
1,1084055,2647117,bitpay.com,1
